In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
BATCH_SIZE = 128
LEARNING_RATE = 0.01
WEIGHT_DECAY = 1e-4
EPOCHS = 151
CONFIDENCE_THRESHOLD = 0.99  # Confidence threshold for high-confidence predictions

# Data augmentation and normalization for training and testing
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Load CIFAR-100 dataset
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Define ResNet block
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# Define ResNet architecture
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=100):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

def resnet101():
    return ResNet(BasicBlock, [3, 4, 23, 3])

# Training function
def train(model, train_loader, optimizer, criterion, epochs, device):
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100.0*correct/total:.2f}%")

# Initialize ResNet-101 and optimizer
model = resnet101()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss()

# Train the model
train(model, train_loader, optimizer, criterion, EPOCHS, device)

# Save the trained model
torch.save(model.state_dict(), 'resnet101_cifar100.pth')
print("Model saved successfully!")

# Evaluate the model on test data
def test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    high_confidence_correct = 0
    high_confidence_total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            confidence, predicted = probabilities.max(1)

            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # High confidence predictions
            high_confidence_mask = confidence > CONFIDENCE_THRESHOLD
            high_confidence_total += high_confidence_mask.sum().item()
            high_confidence_correct += (predicted[high_confidence_mask] == targets[high_confidence_mask]).sum().item()

    test_accuracy = 100.0 * correct / total
    if high_confidence_total > 0:
        high_confidence_accuracy = 100.0 * high_confidence_correct / high_confidence_total
    else:
        high_confidence_accuracy = 0.0

    print(f"Test Accuracy: {test_accuracy:.2f}%")
    print(f"High-confidence Predictions: {high_confidence_total}, Percentage Correct High-confidence Predictions: {high_confidence_accuracy:.2f}%")

# Test the model and check high-confidence predictions
test(model, test_loader, device)


Using device: cuda


100%|██████████| 169001437/169001437 [00:02<00:00, 68182663.64it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/151], Loss: 4.3830, Accuracy: 3.46%
Epoch [2/151], Loss: 3.8521, Accuracy: 9.24%
Epoch [3/151], Loss: 3.4939, Accuracy: 15.55%
Epoch [4/151], Loss: 3.1445, Accuracy: 22.19%
Epoch [5/151], Loss: 2.8039, Accuracy: 28.81%
Epoch [6/151], Loss: 2.5164, Accuracy: 34.45%
Epoch [7/151], Loss: 2.2642, Accuracy: 39.84%
Epoch [8/151], Loss: 2.0703, Accuracy: 44.03%
Epoch [9/151], Loss: 1.9051, Accuracy: 47.78%
Epoch [10/151], Loss: 1.7600, Accuracy: 51.04%
Epoch [11/151], Loss: 1.6495, Accuracy: 53.82%
Epoch [12/151], Loss: 1.5441, Accuracy: 56.25%
Epoch [13/151], Loss: 1.4503, Accuracy: 58.60%
Epoch [14/151], Loss: 1.3709, Accuracy: 60.71%
Epoch [15/151], Loss: 1.2876, Accuracy: 62.97%
Epoch [16/151], Loss: 1.1948, Accuracy: 65.11%
Epoch [17/151], Loss: 1.1291, Accuracy: 66.70%
Epoch [18/151], Loss: 1.0725, Accuracy: 68.15%
Epoch [19/151], Loss: 1.0234, Accuracy: 69.36%
Epoch [20/151], Loss: 0.9628